In [ ]:
# !pip install openai==1.78.1 numpy==2.0.2 pandas==2.2.2 matplotlib==3.10.0

# 프롬프트 엔지니어링 실습

## 다룰 기법들
1. **Zero-shot**: 예시 없이 바로 문제 해결
2. **Few-shot**: 몇 가지 예시로 패턴 학습
3. **Chain-of-Thought (CoT)**: 단계별 추론 과정 명시
4. **Self-Consistency**: 여러 번 풀어서 일관된 답 선택

In [ ]:
# API 키 설정 (3.1.2에서 설명한 방법 사용)
try:
    from google.colab import userdata   
    api_key = userdata.get('OPENAI_API_KEY')
    print("Colab Secrets에서 API 키를 성공적으로 불러왔습니다.")
except:
    import getpass
    api_key = getpass.getpass("OpenAI API 키를 입력하세요: ")
    print("API 키가 입력되었습니다.")

In [ ]:
from openai import OpenAI
import time, re, json
from collections import Counter

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=api_key)
model = "gpt-4o-mini" 

# 기본 함수들
def get_response(prompt, temperature=0, max_tokens=5000):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

## 1. Zero-shot 프롬프트

어떤 예시도 제공하지 않고 모델에게 직접 문제를 해결하도록 지시하는 가장 기본적인 방법입니다.
모델의 사전 지식과 일반화 능력에만 의존합니다.

In [ ]:
# Zero-shot 예제 1: 중국 당나라 한시 스타일 크리스마스 캐롤
problem_1 = """중국 당나라 때 한시 스타일로 크리스마스 캐롤을 만드세요. 한자 아래에 음독을 추가하세요.
"""

response_1 = get_response(problem_1)
print(response_1)

In [ ]:
# Zero-shot 예제 2: 알파벳 세기
# 정답은 12개입니다.
problem_2 = """다음 문장에는 알파벳 b가 몇 개 있나요?
betty and bobb brought back bright blue balloons from the big beach bazzar.
"""

response_2 = get_response(problem_2)
print(response_2)

In [ ]:
# Zero-shot 예제 3: Zero-shot 분류
problem_3 = """다음 문장들을 날씨로 표현해주세요:

1. "오늘 시험을 망쳤어요"
2. "새로운 직장에서 첫 출근을 해요"
3. "오랜만에 친구들과 만났어요"
4. "연인과 헤어졌어요"
5. "로또에 당첨됐어요"
"""

response_3 = get_response(problem_3)
response_3

## 2. Few-shot 프롬프트

몇 가지 예시를 제공하여 모델이 패턴을 학습하도록 하는 방법입니다.
Zero-shot보다 일관성 있는 결과를 얻을 수 있습니다.

In [ ]:
# Few-shot 예제 1: 문장을 날씨로 표현하기 (분류 태깅)
few_shot_prompt_1 = """다음 예시를 참고하여 문장들을 날씨로 표현해주세요:

예시:
입력: "승진 소식을 들었어요."
출력: 맑음

입력: "시험에 떨어졌어요."  
출력: 비

입력: "새로운 도전을 앞두고 있어요."
출력: 안개

이제 다음 문장들을 같은 방식으로 표현해주세요:
1. "몸이 많이 아파요."
2. "충격적인 소식을 들었어요"
3. "시험에 합격했어요."
4. "친구와 대판 싸웠어요."
"""

few_shot_response_1 = get_response(few_shot_prompt_1)
print(few_shot_response_1)

In [ ]:
# Few-shot 예제 2: 음식 재료 JSON 형식 출력
few_shot_prompt_2 = """다음 예시를 참고하여 음식의 재료를 JSON 형식으로 정리하세요:

예시:
입력: "김치찌개"
출력:```json
{"주재료": ["김치", "돼지고기"],
"부재료": ["두부", "파", "양파"],
"조미료": ["고춧가루", "마늘", "생강", "소금"]}
```
입력: "비빔밥"
출력:```json
{"주재료": ["밥", "고사리", "도라지", "시금치", "당근", "콩나물"],
"부재료": ["계란", "쇠고기"],
"조미료": ["고추장", "참기름", "마늘", "설탕"]}
```
이제 다음 순두부찌개를 같은 방식으로 분류해주세요:
"""

few_shot_response_2 = get_response(few_shot_prompt_2)
print(few_shot_response_2)

## 3. Chain-of-Thought (CoT) 프롬프트

단계별 추론 과정을 명시적으로 요구하여 복잡한 문제의 정확도를 크게 향상시키는 방법입니다.
"단계별로 생각해보세요" 같은 지시를 통해 모델이 논리적 사고 과정을 거치도록 유도합니다.

In [ ]:
# CoT 예제 1: 조합과 순열 문제 (정답=10, 60)
cot_prompt_1 = """다음 문제를 단계별로 풀어주세요:

문제: 5명의 학생 중에서 3명을 선발하는 방법의 수는 몇 가지인가요? 그리고 이 5명 중에서 회장 1명, 부회장 1명, 총무 1명을 뽑는 방법의 수는 몇 가지인가요?

해결 과정:
1. 첫 번째 문제 분석
2. 적절한 공식 선택 및 적용
3. 두 번째 문제 분석
4. 적절한 공식 선택 및 적용
5. 두 결과 비교 및 해석

각 단계를 자세히 설명하면서 계산해주세요.
"""

cot_response_1 = get_response(cot_prompt_1)
print(cot_response_1)

In [ ]:
# CoT 예제 2: 기하 문제 (정답=5√3)
cot_prompt_2 = """다음 문제를 단계별로 풀어주세요:

문제: 반지름이 5cm인 원에 내접하는 정삼각형의 한 변의 길이를 구하세요.

해결 과정:
1. 문제 상황 이해
2. 좌표계 설정 또는 기하학적 관계 파악
3. 정삼각형의 성질 활용
4. 삼각함수 또는 피타고라스 정리 적용
5. 최종 답 계산 및 검증

각 단계를 차근 차근 분석하여 푸세요. 자세한 계산 과정을 보여주세요.
"""

cot_response_2 = get_response(cot_prompt_2)
print(cot_response_2)

In [ ]:
# CoT 예제 3: 알파벳 개수 세기 (정답=12)
cot_prompt_3 = """다음 문장에서 알파벳 'b'의 개수를 단계별로 세어주세요:

"betty and bobb brought back bright blue balloons from the big beach bazzar."

단계별 추론 과정:
1. 문장을 단어별로 나누기
2. 각 단어에서 'b' 찾기
3. 대소문자 구분하여 정확히 카운트
4. 최종 합계 계산

각 단계를 자세히 설명하면서 답을 구해주세요.
"""

cot_response_3 = get_response(cot_prompt_3)
print(cot_response_3)

## 4. Self-Consistency

같은 문제를 여러 번 다른 방식으로 풀어보고, 가장 일관된 답을 선택하는 방법입니다.
Chain-of-Thought의 안정성을 높이는 기법입니다.

In [ ]:
def self_consistency(k, cot_prompt):
    
    sc_prompt = '''다음 문제를 단계별로 풀고, 반드시 다음 JSON 형식으로 답변하세요:
{"reasoning": "단계별 풀이 과정", "answer": 최종_숫자_답}'''
    
    prompt = sc_prompt + cot_prompt
    
    extracted_answers = []
    
    print(f"Self-Consistency: {k}번 시도하여 가장 자주 등장한 답을 찾습니다.\n")
    
    for k in range(k):
        print(f"시도 {k+1}:")
        response = get_response(prompt, temperature=0.3)
        
        print(response[:200] + "...\n")
        
        # JSON 답변에서 answer 추출
        try:
            # JSON 패턴 찾기
            json_match = re.search(r'\{[^}]*"answer"[^}]*\}', response)
            if json_match:
                json_str = json_match.group()
                parsed = json.loads(json_str)
                answer = parsed.get("answer")
                if answer is not None:
                    extracted_answers.append(answer)
                    print(f"추출된 답: {answer}")
                else:
                    print("답 추출 실패: answer 키 없음")
            else:
                print("답 추출 실패: JSON 형식 없음")
        except json.JSONDecodeError:
            print("답 추출 실패: JSON 파싱 오류")
        
        time.sleep(1)
    
    print(f"\n전체 추출된 답들: {extracted_answers}")
    
    if extracted_answers:
        answer_counts = Counter(extracted_answers)
        most_common = answer_counts.most_common(1)[0]
        
        print(f"\n답변 빈도:")
        for answer, count in answer_counts.most_common():
            print(f"  {answer}: {count}번")
        
        print(f"\n다수결 결과: {most_common[0]} ({most_common[1]}/{len(extracted_answers)}번)")
        return most_common[0]
    else:
        print("\n유효한 답을 추출하지 못했습니다.")
        return None


In [ ]:
result = self_consistency(5, cot_prompt_3)